In [16]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

## About Data

Twitter has become an important communication channel in times of emergency.
The ubiquitousness of smartphones enables people to announce an emergency they’re observing in real-time. Because of this, more agencies are interested in programatically monitoring Twitter (i.e. disaster relief organizations and news agencies).

But, it’s not always clear whether a person’s words are actually announcing a disaster. Take this example:
![image.png](assets/tweet_screenshot.png)

The author explicitly uses the word “ABLAZE” but means it metaphorically. This is clear to a human right away, especially with the visual aid. But it’s less clear to a machine.

-------
Columns: 

id - a unique identifier for each tweet

text - the text of the tweet

location - the location the tweet was sent from (may be blank)

keyword - a particular keyword from the tweet (may be blank)

target - in train.csv only, this denotes whether a tweet is about a real disaster (1) or not (0)


## Data Prepration

### Reading data

In [17]:
df_train = pd.read_csv(r'data\train.csv')
test = pd.read_csv(r'data\test.csv')

### Investigating the dataset

In [18]:
def background_color(value):
    if isinstance(value, str):
        return 'background-color: #a6c0ed'
    return ''

def show_df(df_train):
    print('shape'.center(30,'_'))
    display(df_train)

    print('head'.center(30,'_'))
    display(df_train.head().style.background_gradient(cmap='Blues'))

    print('tail'.center(30,'_'))
    display(df_train.tail().style.background_gradient(cmap='Blues'))

    print('info'.center(30,'_')+'\n')
    display(df_train.info())

    print('describe_continuous'.center(30,'_'))
    display(df_train.describe().T.style.background_gradient(cmap = 'Blues'))

    print('describe_categorical'.center(30,'_'))
    display(df_train.describe(include='object').T.style.background_gradient(cmap='Blues'))

    print('null_values_percent'.center(30,'_'))
    display((df_train.isna().sum() / len(df_train) * 100).sort_values(ascending=False))
show_df(df_train)

____________shape_____________


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


_____________head_____________


,id,keyword,location,text,target
0,1,nan,nan,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,nan,nan,Forest fire near La Ronge Sask. Canada,1
2,5,nan,nan,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,nan,nan,"13,000 people receive #wildfires evacuation orders in California",1
4,7,nan,nan,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


_____________tail_____________


...

,count,mean,std,min,25%,50%,75%,max
id,7613.000000,5441.934848,3137.116090,1.000000,2734.000000,5408.000000,8146.000000,10873.000000
target,7613.000000,0.429660,0.495060,0.000000,0.000000,0.000000,1.000000,1.000000


_____describe_categorical_____


,count,unique,top,freq
keyword,7552,221,fatalities,45
location,5080,3341,USA,104
text,7613,7503,11-Year-Old Boy Charged With Manslaughter of Toddler: Report: An 11-year-old boy has been charged with manslaughter over the fatal sh...,10


_____null_values_percent______


location    33.272035
keyword      0.801261
id           0.000000
text         0.000000
target       0.000000
dtype: float64

### Preprocessing

#### Removing URL's from the dataset

In [19]:
import re
import string

def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)


In [20]:
print("Text Before:\n", df_train.text[32])
print("Text After:\n",remove_URL(df_train.text[32]))

Text Before:
 We always try to bring the heavy. #metal #RT http://t.co/YAo1e0xngw
Text After:
 We always try to bring the heavy. #metal #RT 


#### Removing punctuations from the dataset

In [21]:
def remove_punct(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [22]:
print("Text Before:\n", df_train.text[2])
print("Text After:\n",remove_punct(df_train.text[2]))

Text Before:
 All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected
Text After:
 All residents asked to shelter in place are being notified by officers No other evacuation or shelter in place orders are expected


#### Removing stop words

In [23]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PrinceEGY\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [24]:
# Stop words example
stopwords.words("english")[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [25]:
def remove_stopwords(text):
    stop = set(stopwords.words("english"))
    
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)

In [26]:
print("Text Before:\n", df_train.text[2])
print("Text After:\n",remove_stopwords(df_train.text[2]))

Text Before:
 All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected
Text After:
 residents asked 'shelter place' notified officers. evacuation shelter place orders expected


#### Now let's apply our cleaning methods

In [27]:
df_train["text"] = df_train.text.map(remove_URL) # map(lambda x: remove_URL(x))
df_train["text"] = df_train.text.map(remove_punct)
df_train["text"] = df_train.text.map(remove_stopwords)
df_train["text"]

0            deeds reason earthquake may allah forgive us
1                   forest fire near la ronge sask canada
2       residents asked shelter place notified officer...
3       13000 people receive wildfires evacuation orde...
4       got sent photo ruby alaska smoke wildfires pou...
                              ...                        
7608    two giant cranes holding bridge collapse nearb...
7609    ariaahrary thetawniest control wild fires cali...
7610                      m194 0104 utc5km volcano hawaii
7611    police investigating ebike collided car little...
7612    latest homes razed northern california wildfir...
Name: text, Length: 7613, dtype: object

-------------

In [14]:
from collections import Counter

# Count unique words
def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count


counter = counter_word(df_train.text)

In [33]:
vocab_cointer = len(counter)
vocab_cointer

17971

In [15]:
counter

Counter({'deeds': 2,
         'reason': 20,
         'earthquake': 50,
         'may': 88,
         'allah': 9,
         'forgive': 2,
         'us': 164,
         'forest': 65,
         'fire': 250,
         'near': 54,
         'la': 25,
         'ronge': 1,
         'sask': 1,
         'canada': 11,
         'residents': 8,
         'asked': 9,
         'shelter': 6,
         'place': 26,
         'notified': 1,
         'officers': 8,
         'evacuation': 50,
         'orders': 11,
         'expected': 15,
         '13000': 4,
         'people': 196,
         'receive': 2,
         'wildfires': 11,
         'california': 117,
         'got': 112,
         'sent': 13,
         'photo': 41,
         'ruby': 1,
         'alaska': 6,
         'smoke': 48,
         'pours': 1,
         'school': 66,
         'rockyfire': 4,
         'update': 37,
         'hwy': 9,
         '20': 26,
         'closed': 20,
         'directions': 1,
         'due': 31,
         'lake': 14,
         'co

In [16]:
counter.most_common(5)

[('like', 345), ('im', 299), ('amp', 298), ('fire', 250), ('get', 229)]